In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import string
import json
import re
from sklearn.utils import shuffle
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sklearn
from sklearn.metrics import confusion_matrix
####
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model

from sklearn.linear_model import LogisticRegression
import sklearn.model_selection as model_selection

from sklearn import svm
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

In [2]:
# open training dataset and parse recipes

with open('train.json') as cooking_file:  
    
    data = json.load(cooking_file)
    

In [3]:
#COUNTVECTORIZER
#removing punctuations and spaces before fixing the data and keep them all in a dict  (italian, mexican)

punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 

            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.replace(' ', '').lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list


clean_italian_cuisine_data = gather_recipes(data, 'italian')
clean_mexican_cuisine_data = gather_recipes(data, 'mexican')

clean_filipino_cuisine_data = gather_recipes(data, 'filipino')
clean_british_cuisine_data = gather_recipes(data, 'british')
clean_moroccan_cuisine_data = gather_recipes(data, 'moroccan')

In [4]:
#creating dfs  (italian, mexican)

italian_df = DataFrame(clean_italian_cuisine_data)

mexican_df = DataFrame(clean_mexican_cuisine_data)


#creating dfs (filipino, british, moroccan )

filipino_df = DataFrame(clean_filipino_cuisine_data) 

british_df = DataFrame(clean_british_cuisine_data)

moroccan_df = DataFrame(clean_moroccan_cuisine_data)


In [5]:
#fixing data for CountVectorizer (italian, mexican)

it_unique_ids = []
it_ingredients = []
it_grouped = italian_df.groupby('id')

for ids, ing in it_grouped:
    
    it_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    it_ingredients.append(row)

    
mex_unique_ids = []
mex_ingredients = []
mex_grouped = mexican_df.groupby('id')

for ids, ing in mex_grouped:
    
    mex_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    mex_ingredients.append(row)
    
#fixing data for CountVectorizer (moroccan, filipino, british)

moroccan_unique_ids = []
moroccan_ingredients = []

moroccan_grouped = moroccan_df.groupby('id')

for ids, ing in moroccan_grouped:
    
    moroccan_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    moroccan_ingredients.append(row)


filipino_unique_ids = []
filipino_ingredients = []

filipino_grouped = filipino_df.groupby('id')

for ids, ing in filipino_grouped:
    
    filipino_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    filipino_ingredients.append(row)


british_unique_ids = []
british_ingredients = []

british_grouped = british_df.groupby('id')

for ids, ing in british_grouped:
    
    british_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    british_ingredients.append(row)

In [6]:
#using CountVectorizer  (italian, mexican) and adding one column Cuisine

it_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
it_array = it_vectorizer.fit_transform(it_ingredients)
it_array = it_array.toarray()
it_df = DataFrame(it_array, columns = it_vectorizer.get_feature_names(), index = it_unique_ids)
it_df['Cuisine'] = 0 #italian
it_df.to_csv('it_cleandata.csv', index_label = 'ID')

mex_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex_array = mex_vectorizer.fit_transform(mex_ingredients)
mex_array = mex_array.toarray()
mex_df = DataFrame(mex_array, columns = mex_vectorizer.get_feature_names(), index = mex_unique_ids)
mex_df['Cuisine'] = 1 #mexican
mex_df.to_csv('mex_cleandata.csv', index_label = 'ID')

#using CountVectorizer (moroccan, filipino, british)

moroccan_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
moroccan_array = moroccan_vectorizer.fit_transform(moroccan_ingredients)
moroccan_array = moroccan_array.toarray()
moroccan_df = DataFrame(moroccan_array, columns = moroccan_vectorizer.get_feature_names(), index = moroccan_unique_ids)
moroccan_df['Cuisine'] = 0 #moroccan
moroccan_df.to_csv('moroccan_cleandata.csv', index_label = 'ID')

british_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
british_array = british_vectorizer.fit_transform(british_ingredients)
british_array = british_array.toarray()
british_df = DataFrame(british_array, columns = british_vectorizer.get_feature_names(), index = british_unique_ids)
british_df['Cuisine'] = 1 #british
british_df.to_csv('british_cleandata.csv', index_label = 'ID')

filipino_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
filipino_array = filipino_vectorizer.fit_transform(filipino_ingredients)
filipino_array = filipino_array.toarray()
filipino_df = DataFrame(filipino_array, columns = filipino_vectorizer.get_feature_names(), index = filipino_unique_ids)
filipino_df['Cuisine'] = 2 #filipino
filipino_df.to_csv('filipino_cleandata.csv', index_label = 'ID')

In [7]:
#putting mexican and italian in one DataFrame and shuffle

df = pd.DataFrame(it_df)
df = df.append(mex_df, sort = False)
df.fillna(0, inplace = True)
df = df.astype('int64')

df = shuffle(df)

#putting moroccan, filipino, british in one DataFrame and shuffle

df_ = pd.DataFrame(moroccan_df)
df_ = df_.append(filipino_df, sort = False)
df_ = df_.append(british_df, sort = False)
df_.fillna(0, inplace = True)
df_ = df_.astype('int64')

df_ = shuffle(df_)

In [8]:
#TFIDFVECTORIZER
#removing punctuations before fixing the data and keep them all in a dict  (italian, mexican)

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 
            
            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list

clean2_italian_cuisine_data = gather_recipes(data, 'italian')
clean2_mexican_cuisine_data = gather_recipes(data, 'mexican')

In [9]:
#creating dfs2  (italian, mexican)

italian2_df = DataFrame(clean2_italian_cuisine_data)

mexican2_df = DataFrame(clean2_mexican_cuisine_data)

In [10]:
#fixing data for TfidfVectorizer (italian, mexican)

it2_unique_ids = []
it2_ingredients = []
it2_grouped = italian2_df.groupby('id')

for ids, ing in it2_grouped:
    
    it2_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    it2_ingredients.append(row)

    
mex2_unique_ids = []
mex2_ingredients = []
mex2_grouped = mexican2_df.groupby('id')

for ids, ing in mex2_grouped:
    
    mex2_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    mex2_ingredients.append(row)


In [11]:
#using TfidfVectorizer  (italian, mexican) and adding one column Cuisine

it2_vectorizer = TfidfVectorizer(analyzer = 'word', binary = True)
it2_array = it2_vectorizer.fit_transform(it2_ingredients)
it2_array = it2_array.toarray()
it2_df = DataFrame(it2_array, columns = it2_vectorizer.get_feature_names(), index = it2_unique_ids)
it2_df['Cuisine'] = 0 #italian
it2_df.to_csv('it2_cleandata.csv', index_label = 'ID')

mex2_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex2_array = mex2_vectorizer.fit_transform(mex2_ingredients)
mex2_array = mex2_array.toarray()
mex2_df = DataFrame(mex2_array, columns = mex2_vectorizer.get_feature_names(), index = mex2_unique_ids)
mex2_df['Cuisine'] = 1 #mexican
mex2_df.to_csv('mex2_cleandata.csv', index_label = 'ID')

In [12]:
#putting mexican and italian in one DataFrame and shuffle

df2 = pd.DataFrame(it2_df)
df2 = df2.append(mex2_df, sort = False)
df2.fillna(0, inplace = True)

df2 = shuffle(df2)

In [13]:
# Multiple Logistic Regression 
lr_m = linear_model.LogisticRegression()

# Support Vector Machines
svm_m = svm.LinearSVC(random_state=0)
#Decision Trees
dtree = tree.DecisionTreeClassifier()

#k-NN
knn = KNeighborsClassifier(n_neighbors = 3)

# Naive Bayes
gnb = MultinomialNB()

In [14]:
print('************ CountVectorizer ITALIAN-MEXICAN ************')
X = df.loc[:, df.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df['Cuisine'] #just the ids in a list

************ CountVectorizer ITALIAN-MEXICAN ************
(14276, 3976)


- 1. ITALIAN MEXICAN CUISINES CLASSIFIERS (COUNTVECTORIZER)

In [15]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****

scores: {'fit_time': array([2.43487   , 2.13762808, 2.15325689, 2.04517722, 2.16187716]), 'score_time': array([0.02811265, 0.02783132, 0.02697802, 0.02757406, 0.0270617 ]), 'test_accuracy': array([0.96848739, 0.97022767, 0.96497373, 0.96707531, 0.96602452]), 'test_precision_weighted': array([0.96848263, 0.97041398, 0.96521523, 0.96715427, 0.96625507]), 'test_recall_weighted': array([0.96848739, 0.97022767, 0.96497373, 0.96707531, 0.96602452]), 'test_f1_weighted': array([0.9684807 , 0.97018936, 0.96492098, 0.9670456 , 0.96597518])}

mean score test accuracy: 0.9673577241754845

mean score test precision weighted: 0.9675042361275435

mean score test recall weighted: 0.9673577241754845

mean score test f1-measure weighted: 0.9673223631338749
Confusion Matrix: [[7674  164]
 [ 302 6136]]


In [16]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****

scores: {'fit_time': array([0.54373312, 0.51653695, 0.50281143, 0.49505305, 0.4967463 ]), 'score_time': array([0.0283587 , 0.02745819, 0.02673483, 0.02708507, 0.0290103 ]), 'test_accuracy': array([0.96113445, 0.96252189, 0.96182137, 0.96532399, 0.96602452]), 'test_precision_weighted': array([0.96114459, 0.96251457, 0.96184237, 0.96536179, 0.96603364]), 'test_recall_weighted': array([0.96113445, 0.96252189, 0.96182137, 0.96532399, 0.96602452]), 'test_f1_weighted': array([0.96113845, 0.96251525, 0.96179738, 0.96529964, 0.966028  ])}

mean score test accuracy: 0.9633652445216265

mean score test precision weighted: 0.9633793907156036

mean score test recall weighted: 0.9633652445216265

mean score test f1-measure weighted: 0.9633557438695888
Confusion Matrix: [[7594  244]
 [ 279 6159]]


In [17]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****Decision-Tree mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Decision-Tree mean scores****

scores: {'fit_time': array([16.54474831, 11.69933081, 15.40012932, 16.61919403, 10.3174901 ]), 'score_time': array([0.02771401, 0.02589655, 0.02651834, 0.02693439, 0.02588987]), 'test_accuracy': array([0.94187675, 0.94956217, 0.93660245, 0.94255692, 0.93485114]), 'test_precision_weighted': array([0.94187675, 0.94957008, 0.93664391, 0.94254428, 0.93491762]), 'test_recall_weighted': array([0.94187675, 0.94956217, 0.93660245, 0.94255692, 0.93485114]), 'test_f1_weighted': array([0.94187675, 0.94956566, 0.93661751, 0.9425277 , 0.93487283])}

mean score test accuracy: 0.9410898860419824

mean score test precision weighted: 0.941110530992076

mean score test recall weighted: 0.9410898860419824

mean score test f1-measure weighted: 0.9410920900113094
Confusion Matrix: [[7419  419]
 [ 411 6027]]


In [18]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****

scores: {'fit_time': array([4.35255456, 4.29130149, 4.35211849, 4.22300029, 4.21780539]), 'score_time': array([172.85635304, 173.75622177, 172.31456304, 172.29213428,
       172.3307631 ]), 'test_accuracy': array([0.89040616, 0.89597198, 0.87810858, 0.87880911, 0.87250438]), 'test_precision_weighted': array([0.89127536, 0.89701345, 0.87805634, 0.87918367, 0.87241041]), 'test_recall_weighted': array([0.89040616, 0.89597198, 0.87810858, 0.87880911, 0.87250438]), 'test_f1_weighted': array([0.88995742, 0.89552032, 0.87792502, 0.87842759, 0.87242975])}

mean score test accuracy: 0.8831600415998272

mean score test precision weighted: 0.8835878450724776

mean score test recall weighted: 0.8831600415998272

mean score test f1-measure weighted: 0.8828520202418714
Confusion Matrix: [[7158  680]
 [ 988 5450]]


In [19]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****

scores: {'fit_time': array([0.51133418, 0.51180577, 0.50943851, 0.50872707, 0.51019549]), 'score_time': array([0.06491613, 0.02915382, 0.02869201, 0.0294683 , 0.02944326]), 'test_accuracy': array([0.96673669, 0.96952715, 0.96357268, 0.96357268, 0.96287215]), 'test_precision_weighted': array([0.96679036, 0.96986848, 0.9639082 , 0.96372807, 0.96307815]), 'test_recall_weighted': array([0.96673669, 0.96952715, 0.96357268, 0.96357268, 0.96287215]), 'test_f1_weighted': array([0.96671063, 0.96947438, 0.96350793, 0.96352767, 0.96281986])}

mean score test accuracy: 0.9652562706343483

mean score test precision weighted: 0.9654746511163024

mean score test recall weighted: 0.9652562706343483

mean score test f1-measure weighted: 0.9652080950271873
Confusion Matrix: [[7678  160]
 [ 336 6102]]


In [20]:
print('************ TfidfVectorizer ITALIAN-MEXICAN ************')
X = df2.loc[:, df2.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df2['Cuisine'] #just the ids in a list

************ TfidfVectorizer ITALIAN-MEXICAN ************
(14276, 2006)


- 2. ITALIAN MEXICAN CUISINES CLASSIFIERS (TFIDFVECTORIZER)

In [21]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****

scores: {'fit_time': array([0.54130411, 0.50045037, 0.51006675, 0.48938179, 0.51969409]), 'score_time': array([0.00981164, 0.00975919, 0.00973296, 0.00974083, 0.00989151]), 'test_accuracy': array([0.99404762, 0.99264448, 0.99474606, 0.99369527, 0.99264448]), 'test_precision_weighted': array([0.99411146, 0.99274175, 0.99479588, 0.99376683, 0.99274169]), 'test_recall_weighted': array([0.99404762, 0.99264448, 0.99474606, 0.99369527, 0.99264448]), 'test_f1_weighted': array([0.99404389, 0.99263873, 0.99474319, 0.99369106, 0.9926387 ])}

mean score test accuracy: 0.9935555833541823

mean score test precision weighted: 0.993631521983805

mean score test recall weighted: 0.9935555833541823

mean score test f1-measure weighted: 0.9935511146866471
Confusion Matrix: [[7838    0]
 [  92 6346]]


In [22]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****

scores: {'fit_time': array([0.19384384, 0.19438696, 0.19334221, 0.19095731, 0.19285488]), 'score_time': array([0.01016283, 0.00978446, 0.00991774, 0.00995016, 0.0098083 ]), 'test_accuracy': array([0.99544818, 0.99474606, 0.99614711, 0.99649737, 0.9943958 ]), 'test_precision_weighted': array([0.99548561, 0.99479588, 0.99616465, 0.99651957, 0.9944524 ]), 'test_recall_weighted': array([0.99544818, 0.99474606, 0.99614711, 0.99649737, 0.9943958 ]), 'test_f1_weighted': array([0.99544603, 0.99474319, 0.99614587, 0.99649611, 0.99439249])}

mean score test accuracy: 0.995446903805305

mean score test precision weighted: 0.9954836210973221

mean score test recall weighted: 0.995446903805305

mean score test f1-measure weighted: 0.9954447386842503
Confusion Matrix: [[7837    1]
 [  64 6374]]


In [23]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****Decision-Tree mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Decision-Tree mean scores****

scores: {'fit_time': array([2.92740965, 2.92198825, 2.9885025 , 3.05996656, 2.86281371]), 'score_time': array([0.014709  , 0.01449299, 0.01541448, 0.01520777, 0.01592159]), 'test_accuracy': array([0.9982493 , 0.99859895, 0.99824869, 0.99789842, 0.99789842]), 'test_precision_weighted': array([0.99825486, 0.99860252, 0.99825426, 0.99790179, 0.99790643]), 'test_recall_weighted': array([0.9982493 , 0.99859895, 0.99824869, 0.99789842, 0.99789842]), 'test_f1_weighted': array([0.99824899, 0.99859875, 0.99824838, 0.99789813, 0.99789798])}

mean score test accuracy: 0.998178756616482

mean score test precision weighted: 0.998183972178969

mean score test recall weighted: 0.998178756616482

mean score test f1-measure weighted: 0.9981784448203082
Confusion Matrix: [[7837    1]
 [  28 6410]]


In [24]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****

scores: {'fit_time': array([2.37413383, 2.31489491, 2.29085636, 2.2969327 , 2.2828486 ]), 'score_time': array([85.80860066, 85.81621575, 85.7117753 , 86.25588083, 88.64099121]), 'test_accuracy': array([0.94082633, 0.93450088, 0.92924694, 0.93765324, 0.92609457]), 'test_precision_weighted': array([0.94658358, 0.94148396, 0.93732614, 0.94400932, 0.9348602 ]), 'test_recall_weighted': array([0.94082633, 0.93450088, 0.92924694, 0.93765324, 0.92609457]), 'test_f1_weighted': array([0.9402619 , 0.93378146, 0.9283789 , 0.93701172, 0.9251245 ])}

mean score test accuracy: 0.9336643904497001

mean score test precision weighted: 0.9408526421684199

mean score test recall weighted: 0.9336643904497001

mean score test f1-measure weighted: 0.9329116970930175
Confusion Matrix: [[7838    0]
 [ 947 5491]]


In [25]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****
scores: {'fit_time': array([0.11841631, 0.10556841, 0.10474229, 0.10502625, 0.10627341]), 'score_time': array([0.0115602 , 0.01059508, 0.01038337, 0.01046515, 0.01044774]), 'test_accuracy': array([0.96428571, 0.96322242, 0.9614711 , 0.96252189, 0.96357268]), 'test_precision_weighted': array([0.96429613, 0.96328558, 0.96159757, 0.96274039, 0.96366238]), 'test_recall_weighted': array([0.96428571, 0.96322242, 0.9614711 , 0.96252189, 0.96357268]), 'test_f1_weighted': array([0.9642675 , 0.96319072, 0.96142677, 0.96246711, 0.96353659])}

mean score test accuracy: 0.9630147610708031

mean score test precision weighted: 0.9631164118420814

mean score test recall weighted: 0.9630147610708031

mean score test f1-measure weighted: 0.9629777378584586
Confusion Matrix: [[7639  199]
 [ 329 6109]]


- 3. MOROCCAN FILIPINO BRITISH CUISINES CLASSIFIERS (COUNTVECTORIZER)

In [26]:
print('********* CountVectorizer MOROCCAN, FILIPINO, BRITISH *********')
X = df_.loc[:, df_.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df_['Cuisine'] #just the ids in a list

********* CountVectorizer MOROCCAN, FILIPINO, BRITISH *********
(2380, 2074)


In [27]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****

scores: {'fit_time': array([0.31670904, 0.25125337, 0.26215076, 0.25146651, 0.25886679]), 'score_time': array([0.00797939, 0.00487256, 0.00488257, 0.00489902, 0.00489998]), 'test_accuracy': array([0.93067227, 0.93487395, 0.93697479, 0.93487395, 0.92226891]), 'test_precision_weighted': array([0.930708  , 0.93526268, 0.9383125 , 0.93522376, 0.92389225]), 'test_recall_weighted': array([0.93067227, 0.93487395, 0.93697479, 0.93487395, 0.92226891]), 'test_f1_weighted': array([0.93068386, 0.93482843, 0.93709376, 0.93482409, 0.92255015])}

mean score test accuracy: 0.9319327731092437

mean score test precision weighted: 0.9326798360359623

mean score test recall weighted: 0.9319327731092437

mean score test f1-measure weighted: 0.9319960591672676
Confusion Matrix: [[774  34  13]
 [ 18 753  33]
 [ 16  48 691]]


In [28]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****

scores: {'fit_time': array([0.05423188, 0.05288363, 0.0509882 , 0.05211639, 0.05363965]), 'score_time': array([0.00494146, 0.00481749, 0.00482297, 0.0049448 , 0.00483489]), 'test_accuracy': array([0.93277311, 0.92647059, 0.93697479, 0.94327731, 0.92436975]), 'test_precision_weighted': array([0.93261239, 0.92718705, 0.93714065, 0.94391322, 0.9256819 ]), 'test_recall_weighted': array([0.93277311, 0.92647059, 0.93697479, 0.94327731, 0.92436975]), 'test_f1_weighted': array([0.93265056, 0.92634114, 0.9370391 , 0.94309959, 0.92470571])}

mean score test accuracy: 0.9327731092436975

mean score test precision weighted: 0.9333070434046942

mean score test recall weighted: 0.9327731092436975

mean score test f1-measure weighted: 0.932767220095743
Confusion Matrix: [[784  24  13]
 [ 24 746  34]
 [ 16  49 690]]


In [29]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****Decision Tree mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Decision Tree mean scores****

scores: {'fit_time': array([0.20144629, 0.1638267 , 0.15767384, 0.1467917 , 0.15117407]), 'score_time': array([0.00531268, 0.005126  , 0.00494361, 0.0048945 , 0.00495648]), 'test_accuracy': array([0.83823529, 0.82142857, 0.84663866, 0.81722689, 0.80252101]), 'test_precision_weighted': array([0.84044051, 0.82103695, 0.84663866, 0.81705326, 0.80463786]), 'test_recall_weighted': array([0.83823529, 0.82142857, 0.84663866, 0.81722689, 0.80252101]), 'test_f1_weighted': array([0.8389445 , 0.82113061, 0.84663866, 0.81706875, 0.80320098])}

mean score test accuracy: 0.8252100840336134

mean score test precision weighted: 0.8259614462814804

mean score test recall weighted: 0.8252100840336134

mean score test f1-measure weighted: 0.8253966990628715
Confusion Matrix: [[696  93  32]
 [ 81 645  78]
 [ 29  93 633]]


In [30]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****

scores: {'fit_time': array([0.23883939, 0.2306757 , 0.23761582, 0.230443  , 0.23148203]), 'score_time': array([2.27386808, 2.35723877, 2.30405426, 2.2652657 , 2.28184795]), 'test_accuracy': array([0.82142857, 0.79621849, 0.78151261, 0.80672269, 0.81302521]), 'test_precision_weighted': array([0.82715918, 0.79635473, 0.7863361 , 0.8072959 , 0.81314749]), 'test_recall_weighted': array([0.82142857, 0.79621849, 0.78151261, 0.80672269, 0.81302521]), 'test_f1_weighted': array([0.82084686, 0.79605915, 0.78123804, 0.80678095, 0.81301914])}

mean score test accuracy: 0.803781512605042

mean score test precision weighted: 0.8060586800338225

mean score test recall weighted: 0.803781512605042

mean score test f1-measure weighted: 0.8035888284629623
Confusion Matrix: [[653  67 101]
 [ 92 634  78]
 [ 44  85 626]]


In [31]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****

scores: {'fit_time': array([0.04877329, 0.04653096, 0.04624033, 0.04695749, 0.04790783]), 'score_time': array([0.00528336, 0.00479269, 0.00493193, 0.00487447, 0.00494599]), 'test_accuracy': array([0.91386555, 0.92016807, 0.92016807, 0.93277311, 0.92647059]), 'test_precision_weighted': array([0.91468737, 0.92326225, 0.92222858, 0.9339888 , 0.92837013]), 'test_recall_weighted': array([0.91386555, 0.92016807, 0.92016807, 0.93277311, 0.92647059]), 'test_f1_weighted': array([0.91402941, 0.92017462, 0.92063598, 0.93262485, 0.92685223])}

mean score test accuracy: 0.9226890756302522

mean score test precision weighted: 0.9245074262191073

mean score test recall weighted: 0.9226890756302522

mean score test f1-measure weighted: 0.9228634181488091
Confusion Matrix: [[772  43   6]
 [ 27 747  30]
 [ 15  63 677]]
